**Mastère Spécialisé VADLOM 2025**

**INP ENSEEIHT**

*Authors : Abdelali El Harrani, Abdoulaye Syll*

✉️ *: abdelali.elharrani@etu.toulouse-inp.fr, abdoulaye.syll@etu.toulouse-inp.fr*

# Projet LLM engineering : Data Agent

Ce notebook a pour objectif de développer un agent 'data' capable de répondre à des questions en se basant sur les informations présentes sur des fichiers csv et json.
Le framework utilisé est Langchain.

**Le projet est divisé en deux parties :**

- Création d'un agent csv capables de trouver des informations au sein de fichiers csv fournis
- Tentative non fonctionelle d'implémentation d'un agent RAG capable de trouver des informations dans des fichiers json


## Partie 1 : Agent csv

### I) Preprocessing des données

Les fichiers récupérés proviennent du site : https://data.assemblee-nationale.fr/ <br>
Ils comportent la liste des députés actuels ainsi que la liste des collaborateurs par députés

In [ ]:
import chardet

# Détecter l'encodage du fichier
with open("data/liste_deputes_excel.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # On analyse un échantillon
    encoding = result["encoding"]

print(f"Encodage détecté : {encoding}")

Encodage détecté : ISO-8859-1


In [ ]:
import pandas as pd
df1 = pd.read_csv("data/liste_deputes_excel.csv", encoding=encoding, sep=";")

# Sauvegarder en UTF-8
df1.to_csv("data/liste_deputes_utf8.csv", encoding="utf-8", index=False)

In [ ]:
import pandas as pd
df2 = pd.read_csv("data/liste_collaborateurs_excel.csv", encoding=encoding, sep=";")

# Sauvegarder en UTF-8
df2.to_csv("data/liste_collaborateurs_utf8.csv", encoding="utf-8", index=False)

In [ ]:
#Visualisation de la structure du fichier
df1.head()

,identifiant,Prénom,Nom,Région,Département,Numéro de circonscription,Profession,Groupe politique (complet),Groupe politique (abrégé)
0,795998,Émeline,K/Bidi,Réunion,Réunion,4,Avocate,Gauche Démocrate et Républicaine,GDR
1,796106,Édouard,Bénard,Normandie,Seine-Maritime,3,Collaborateur d'élu,Gauche Démocrate et Républicaine,GDR
2,719412,Sandrine,Le Feur,Bretagne,Finistère,4,Agriculteur-propriétaire exploitant,Ensemble pour la République,EPR
3,841553,Thierry,Tesson,Hauts-de-France,Nord,17,(74) - Ancien cadre,Rassemblement National,RN
4,794946,Antoine,Villedieu,Bourgogne-Franche-Comté,Haute-Saône,1,Policier,Rassemblement National,RN


In [ ]:
#Visualisation de la structure du fichier
df2.head()

,Identifiant du député,Nom du député,Prénom du député,Nom du collaborateur,Prénom du collaborateur
0,PA840773,Fayssat,Olivier,Rojo,Florian
1,PA840773,Fayssat,Olivier,Roland,Damien
2,PA840773,Fayssat,Olivier,Rioult,Olivier
3,PA840773,Fayssat,Olivier,Gregori,Anne-Marie
4,PA793928,Diaz,Edwige,Jacquemin,Hager


### II) Importation du model et instanciation de l'agent

In [ ]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="qwen2.5-coder:32b")

In [ ]:
#!pip install --no-cache-dir --target=packages/tabulate tabulate

In [ ]:
#Décommenter si le package ne peut pas être importé
import sys
sys.path.append("packages/tabulate")

In [ ]:
import tabulate
from langchain_experimental.agents import create_csv_agent

csv_agent = create_csv_agent(
    model,
    'data/liste_deputes_utf8.csv',#Pour le moment,l'agent est testé sur un seul fichier
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
)

/root/valdom_env/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


### III) Evaluation

In [ ]:
csv_agent.run("Combien y a-t-il de députés écologistes?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to check if there is any specific label for "écologistes" (ecologists) in the 'Groupe politique (complet)' or 'Groupe politique (abrégé)' columns. If not directly labeled, we might have to infer based on other information, but given the context, it's likely that the data should explicitly mention ecologists if they are present.

Action: python_repl_ast
Action Input: df['Groupe politique (complet)'].unique()['Gauche Démocrate et Républicaine' 'Ensemble pour la République'
 'Rassemblement National' 'Horizons & Indépendants' 'Écologiste et Social'
 'Socialistes et apparentés'
 'La France insoumise - Nouveau Front Populaire' 'Droite Républicaine'
 'Les Démocrates' 'UDR' 'Libertés, Indépendants, Outre-mer et Territoires'
 'Non inscrit']Thought: From the unique values in the 'Groupe politique (complet)' column, I can see that there is an entry labeled 'Écologiste et Social'. This suggests that there are ecologis

'Il y a 38 députés écologistes dans le dataset.'

In [ ]:
csv_agent.run("Quels sont les députés d'île de France du RN?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe for rows where the Région is "Île-de-France" and the Groupe politique (abrégé) is "RN". Then I will select the relevant columns (Prénom and Nom) to identify the députés.

Action: python_repl_ast
Action Input: df[(df['Région'] == 'Île-de-France') & (df['Groupe politique (abrégé)'] == 'RN')][['Prénom', 'Nom']]Empty DataFrame
Columns: [Prénom, Nom]
Index: []It seems there are no députés from Île-de-France in the Rassemblement National based on the provided dataset. Let's double-check by ensuring that the region names and political group abbreviations are correctly specified.

Action: python_repl_ast
Action Input: df['Région'].unique()['Réunion' 'Normandie' 'Bretagne' 'Hauts-de-France'
 'Bourgogne-Franche-Comté' 'Pays de la Loire' 'Ile-de-France'
 "Provence-Alpes-Côte d'Azur" 'Auvergne-Rhône-Alpes' 'Grand Est'
 'Occitanie' 'Centre-Val de Loire' 'Nouvelle-Aquitaine'
 'Polynésie française

"Les députés d'Île de France du RN sont Julien Limongi, Béatrice Roullaud, Anne Sicard et Nathalie Da Conceicao Carvalho."

In [ ]:
csv_agent.run("Quel département a le plus de circonscriptions")



> Entering new AgentExecutor chain...
Thought: To determine which department has the most unique electoral districts (circonscriptions), I'll need to group the dataframe by both 'Département' and 'Numéro de circonscription', count the occurrences, and then find the department with the highest number of unique districts.

Action: python_repl_ast
Action Input: df.groupby(['Département', 'Numéro de circonscription']).size().reset_index(name='counts').groupby('Département')['counts'].sum().idxmax()NordThe department with the most unique electoral districts (circonscriptions) is "Nord".

Final Answer: Nord

> Finished chain.


'Nord'

In [ ]:
csv_agent.run("Quels sont les députés journalistes")



> Entering new AgentExecutor chain...
Thought: To find out which deputies are journalists, I need to search for the profession "Journaliste" in the "Profession" column of the dataframe. Since the dataset is in French, I will look for "Journaliste" specifically.
Action: python_repl_ast
Action Input: df[df['Profession'] == 'Journaliste']     identifiant         Prénom         Nom              Région  \
93        331973        Pascale         Got  Nouvelle-Aquitaine   
173       309643      Véronique       Besse    Pays de la Loire   
428       719528  Jean-François  Portarrieu           Occitanie   

       Département  Numéro de circonscription   Profession  \
93         Gironde                          5  Journaliste   
173         Vendée                          4  Journaliste   
428  Haute-Garonne                          5  Journaliste   

    Groupe politique (complet) Groupe politique (abrégé)  
93   Socialistes et apparentés                       SOC  
173                Non in

'The deputies who are journalists are Pascale Got, Véronique Besse, and Jean-François Portarrieu.'

In [ ]:
csv_agent.run("Quelle est la profession du député de la 3e circonscription du Nord?")



> Entering new AgentExecutor chain...
Thought: To find the profession of the député from the 17th circoscrption of Nord (which corresponds to the 3rd circoscrption mentioned in the question, but I'll verify that), I need to filter the dataframe for entries where the Département is "Nord" and the Numéro de circonscription is 17. Then I can select the Profession column from the resulting row(s).

Action: python_repl_ast
Action Input: df[(df['Département'] == 'Nord') & (df['Numéro de circonscription'] == 17)]['Profession']3    (74) - Ancien cadre
Name: Profession, dtype: objectThe profession of the député from the 17th circonscription of Nord is "(74) - Ancien cadre". However, this seems to contradict the question's mention of the "3e circonscription du Nord". Let's verify if there is a député in the 3rd circonscription of Nord.

Action: python_repl_ast
Action Input: df[(df['Département'] == 'Nord') & (df['Numéro de circonscription'] == 3)]['Profession']248    (21) - Artisan
Name: Profe

'La profession du député de la 3e circonscription du Nord est "(21) - Artisan".'

### IV) Amélioation de l'agent

Maintenant que l'agent est fonctionnel et répond facilement aux questions posées,
essayons de le challenger avec des questions qui portent sur plusieurs fichiers csv.

In [ ]:
csv_agent2 = create_csv_agent(
    model,
    ['data/liste_deputes_utf8.csv','data/liste_collaborateurs_utf8.csv'], #On ajoute le second fichier
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
)

/root/valdom_env/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [ ]:
csv_agent2.run("Quelle est la région du député dont le collaborateur est Quentin Dellong?")

/tmp/ipykernel_2003518/2944703566.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  csv_agent2.run("Quelle est la région du député dont le collaborateur est Quentin Dellong?")




> Entering new AgentExecutor chain...
Thought: To find the region of the deputy whose collaborator is Quentin Dellong, I need to first identify the deputy associated with this collaborator in `df2` and then use that information to fetch the corresponding region from `df1`.

Action: First, filter df2 to find the row where the Prénom du collaborateur is 'Quentin' and Nom du collaborateur is 'Dellong'.
Action Input: df2[(df2['Prénom du collaborateur'] == 'Quentin') & (df2['Nom du collaborateur'] == 'Dellong')]First, filter df2 to find the row where the Prénom du collaborateur is 'Quentin' and Nom du collaborateur is 'Dellong'. is not a valid tool, try one of [python_repl_ast].Thought: I need to correct my approach to use the `python_repl_ast` tool properly. Let's filter `df2` to find the row where the Prénom du collaborateur is 'Quentin' and Nom du collaborateur is 'Dellong', and then extract the Identifiant du député from that row.

Action: python_repl_ast
Action Input: df2[(df2['Préno

'La région du député dont le collaborateur est Quentin Dellong est Polynésie française.'

In [ ]:
csv_agent2.run("Quelle est le partie politique du député dont le collaborateur est Marie Marteau?")

## Partie 2 : RAG json

Les fichiers ajoutés sont issus du même site et rassemblent des données sur les réunions à l'assemblée.

### I) Prétraitement des données

In [ ]:
import json
from pathlib import Path
from pprint import pprint


file_path='data/JSON_DATA2/RUANR5L17S2025IDC453520.json'
data = json.loads(Path(file_path).read_text())

In [ ]:
from langchain.schema import Document
data = json.loads(Path(file_path).read_text())


# Extraire les informations principales du fichier JSON
reunion_info = data.get('reunion', {})

uid = reunion_info.get('uid', 'Unknown')
time_debut = reunion_info.get('timeStampDebut', 'Unknown')
time_fin = reunion_info.get('timeStampFin', 'Unknown')
lieu = reunion_info.get('lieu', {}).get('libelleLong', 'Unknown')
cycle_de_vie = reunion_info.get('cycleDeVie', {}).get('etat', 'Unknown')
participants = reunion_info.get('participants', {}).get('participantsInternes', {}).get('participantInterne', [])
odj_items = reunion_info.get('ODJ', {}).get('convocationODJ', {}).get('item', [])

# Création d'un seul document avec toutes les informations extraites
page_content = []

# Ajouter les détails de la réunion dans le contenu
page_content.append(f"Réunion UID: {uid}, Debut: {time_debut}, Fin: {time_fin}, Lieu: {lieu}, Cycle de vie: {cycle_de_vie}")

# Ajouter les participants dans le contenu
for i, participant in enumerate(participants, start=1):
    acteur_ref = participant.get('acteurRef', 'Unknown')
    presence = participant.get('presence', 'Inconnu')
    page_content.append(f"Participant {i}: ActeurRef: {acteur_ref}, Présence: {presence}")

# Ajouter les points de l'ordre du jour (ODJ) dans le contenu
for j, item in enumerate(odj_items, start=1):
    page_content.append(f"Point ODJ {j}: {item}")

# Convertir la liste en une chaîne de caractères (chaque élément est une ligne distincte)
full_page_content = "\n".join(page_content)

# Créer un document avec toutes les informations
document = Document(
    page_content=full_page_content,
    metadata={'source': file_path, 'seq_num': 1}
)

# Assurez-vous que vous passez un seul document à la fonction qui attend une liste de documents
documents = [document]

### II) Importation du modèle et création de l'interface RAG

In [ ]:
# Import Ollama models for retrieval-augmented generation
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings

# Initialize a language model for response generation
generation_llm = OllamaLLM(model="qwen2.5-coder:32b")

# Initialize an embedding model for vector-based retrieval (used later)
embedding = OllamaEmbeddings(model="mistral")

In [ ]:
from abc import abstractmethod, ABC
from langchain_core.documents import Document
from typing import List, Optional, Any

class RAGInterface(ABC):
    """
    Abstract class defining a generic RAG system.

    This class ensures that all RAG implementations follow a common structure.
    """

    def __init__(self, name: str, knowledge_db: Optional[Any] = None):
        self.name = name  # Identifier for the RAG system
        self.knowledge_db = knowledge_db  # Storage backend (e.g., a vector database)

    @abstractmethod
    def retrieve(self, query: str) -> List[Document]:
        """
        Retrieve relevant contexts from the knowledge_db based on the query.

        Args:
            query (str): The user query.

        Returns:
            List[Document]: Retrieved document chunks.
        """
        pass

    @abstractmethod
    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generate a response based on the query and retrieved contexts.

        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Relevant document chunks.

        Returns:
            str: The generated response.
        """
        pass


### III) Stratégie de prompt

In [ ]:
from langchain_core.prompts import PromptTemplate

# Define a Structured Prompt for Generation
prompt_template_v0 = """
You are an assistant for question-answering tasks, and you will be processing documents related to meetings of the French National Assembly.
These documents include details about the date, time, participants (along with their attendance), location, and agenda of the meeting. The structure of the meeting data is as follows:
- Date and Time: Début to Fin
- Location: Lieu
- Participants: Each particitpant has an ActeurRef and an indicator of presence : Présent or Absent
- Agenda: ODJ

Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {query}
Context: {retrieved_contexts}
Answer:
"""

# Create a prompt template from the string
gen_prompt_v0 = PromptTemplate.from_template(prompt_template_v0)

### IV) Première tentative avec BM25

In [ ]:
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode

# Define BM25-Based RAG System
class BM25V0RAG(RAGInterface):
    """
    Sparse Retrieval RAG using BM25 and Qdrant.

    - Stores text chunks in Qdrant using BM25 sparse embeddings.
    - Retrieves the top-k relevant chunks based on keyword matching.
    - Uses a language model to generate answers from retrieved contexts.
    """

    def __init__(self, generation_model: OllamaLLM):
        # Initialize BM25 sparse embeddings
        sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25",cache_dir=".")

        # Store documents in Qdrant using sparse retrieval
        knowledge_db = QdrantVectorStore.from_documents(
            documents,  # Pre-processed text chunks
            embedding=OllamaEmbeddings(model="mistral"),  # Not used in this mode, but required
            sparse_embedding=sparse_embeddings,  # BM25 embeddings
            location=":memory:",  # Store in-memory (can be changed to persistent storage)
            retrieval_mode=RetrievalMode.SPARSE,  # Use only sparse retrieval
        )

        # Define model name dynamically
        name = f"bm25_v0_{generation_model.model}"
        super().__init__(name=name, knowledge_db=knowledge_db)

        # Initialize the LLM and retriever
        self.llm = generation_model
        self.retriever = self.knowledge_db.as_retriever(
            search_type="similarity", search_kwargs={"k": 30}
        )
        self.gen_prompt = gen_prompt_v0  # Use the structured prompt

    def retrieve(self, query: str) -> List[Document]:
        """
        Retrieves relevant documents using BM25 sparse retrieval.

        Args:
            query (str): The user query.

        Returns:
            List[Document]: The most relevant document chunks.
        """
        retrieved_contexts = self.retriever.invoke(query)
        return retrieved_contexts

    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generates a response using the retrieved contexts.

        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Retrieved document chunks.

        Returns:
            str: The generated answer.
        """
        # Format retrieved contexts into a single string
        format_retrieved_contexts = "\n".join([rc.page_content for rc in retrieved_contexts])

        # Format the query with the retrieved contexts
        augmented_query = self.gen_prompt.format(
            query=query,
            retrieved_contexts=format_retrieved_contexts
        )

        # Generate the final response
        response = self.llm.invoke(augmented_query)
        return response

In [ ]:
# Initialize the generation model
generation_llm = OllamaLLM(model="qwen2.5-coder:32b")

RAG1 = BM25V0RAG(generation_model=generation_llm)

In [ ]:
query = "Quels sont les ACTEUR REF des personnes absentes lors de le réunion? Répond en me donnant seulement la liste de tout les acteurRef des personnes absentes"

retrieved_context = RAG1.retrieve(query + prompt_template_v0)

answer = RAG1.generate(query + prompt_template_v0, retrieved_context)

In [ ]:
print(retrieved_context)
print(answer)

### V) Deuxième tentative avec un retreiver hybride

In [ ]:
# Define Fusion Algorithms
from qdrant_client import models

rrf = models.FusionQuery(fusion=models.Fusion.RRF)
dbsf = models.FusionQuery(fusion=models.Fusion.DBSF)

In [ ]:
class HYBRIDV1RAG(RAGInterface):
    """
    Hybrid RAG using Distribution-Based Score Fusion (DBSF).
    Combines Sparse (BM25) and Dense (Embeddings) retrieval.
    """

    def __init__(self, generation_model: OllamaLLM):
        # Initialize sparse embeddings
        sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

        # Create Hybrid Knowledge Base
        knowledge_db = QdrantVectorStore.from_documents(
            documents,  # Choose an ingestion pipeline
            embedding=OllamaEmbeddings(model="mistral"),
            sparse_embedding=sparse_embeddings,
            location=":memory:",
            collection_name="deputies_list_hybrid_v1",
            retrieval_mode=RetrievalMode.HYBRID,
        )

        # Initialize RAG Interface
        name = f"hybrid_v1_{generation_model.model}"
        super().__init__(name=name, knowledge_db=knowledge_db)
        self.llm = generation_model

        # Configure retriever with DBSF Fusion
        self.retriever = self.knowledge_db.as_retriever(
            search_type="similarity",
            search_kwargs={"hybrid_fusion": dbsf}  # Apply DBSF fusion
        )

        self.gen_prompt = gen_prompt_v0

    def retrieve(self, query: str) -> List[Document]:
        return self.retriever.invoke(query)

    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        augmented_query = self.gen_prompt.format(query=query, retrieved_contexts=retrieved_contexts)
        return self.llm.invoke(augmented_query)

In [ ]:
RAG2 = HYBRIDV1RAG(generation_model=generation_llm)

In [ ]:
query = "Quels sont les acteurRef des absents?"

retrieved_context = RAG2.retrieve(query)

answer = RAG2.generate(query, retrieved_context)

In [ ]:
print(retrieved_context)
print(answer)